In [1]:
import anndata as ad 

In [2]:
import pickle 
file = pickle.load(open("/work/home/cryoem666/xyf/temp/pycharm/Squidiff/checkpoints/ours_small_test_total2k_gene_new/vocab_state.pkl","rb"))

In [3]:
len(file['cell_line']), len(file['batch']), len(file['perturb'])

(4, 56, 2014)

In [113]:
#squidiff_Data = ad.read_h5ad("/work/home/cryoem666/xyf/temp/pycharm/Squidiff/data/diff_data/diff_train.h5ad")
import toml 
config = toml.load("/work/home/cryoem666/xyf/temp/pycharm/Squidiff/data/replogle/hepg2.toml")
state_Data = ad.read_h5ad(config['datasets']['replogle_proper'])

In [5]:
test_gene_list = config['fewshot']['replogle_proper.hepg2']['val'] + config['fewshot']['replogle_proper.hepg2']['test']

In [111]:
state_Data = state_Data[state_Data.obs['gene'].isin(file['perturb'])]

In [7]:
len(file['perturb']), len(file['batch']), len(file['cell_line'])

(2014, 56, 4)

In [8]:
from sympy import use
import sample_squidiff_ours
sampler = sample_squidiff_ours.sampler(
        model_path = '/work/home/cryoem666/xyf/temp/pycharm/Squidiff/Squidiff/checkpoints/replogle_train_total_vae_2k/model.pt',
        perturb_len = len(file['perturb']),
        batch_len = len(file['batch']),
        cell_line_len = len(file['cell_line']),
        gene_size = 128,
        #gene_size=201,
        diffusion_steps = 1000,
        use_vae=True
    )

load model and diffusion...
{'output_dim': 128, 'num_layers': 3, 'num_channels': 128, 'dropout': 0.0, 'class_cond': False, 'use_checkpoint': False, 'use_scale_shift_norm': True, 'use_fp16': False, 'use_encoder': True, 'state_dataset_config': {'perturb_len': 2014, 'batch_len': 56, 'cell_line_len': 4, 'gene_size': 128, 'output_dim': 128}, 'learn_sigma': False, 'diffusion_steps': 1000, 'noise_schedule': 'linear', 'timestep_respacing': '', 'use_kl': False, 'predict_xstart': False, 'rescale_timesteps': False, 'rescale_learned_sigmas': False, 'data_path': '', 'schedule_sampler': 'uniform', 'lr': 0.0001, 'weight_decay': 0.0, 'lr_anneal_steps': 100000.0, 'batch_size': 16, 'microbatch': -1, 'ema_rate': '0.9999', 'log_interval': 10000.0, 'save_interval': 10000.0, 'resume_checkpoint': '', 'fp16_scale_growth': 0.001, 'use_ddim': True, 'logger_path': '', 'model_path': '/work/home/cryoem666/xyf/temp/pycharm/Squidiff/Squidiff/checkpoints/replogle_train_total_vae_2k/model.pt', 'comb_num': 1, 'drug_dim

In [9]:
import numpy as np 
def one_hot(label, key):
    vec = np.zeros(len(file[key]), dtype=int)
    idx = file[key][label]
    vec[idx] = 1
    return vec

In [9]:
state_Data.obs['perturb_label'] = state_Data.obs['gene'].apply(lambda x: file['perturb'][x])
state_Data.obs['batch_label'] = state_Data.obs['gem_group'].apply(lambda x: file['batch'][x])
state_Data.obs['cell_line_label'] = state_Data.obs['cell_line'].apply(lambda x: file['cell_line'][x])

/tmp/ipykernel_211389/1927203975.py:1: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  state_Data.obs['perturb_label'] = state_Data.obs['gene'].apply(lambda x: file['perturb'][x])


In [10]:
hepg2_pred = ad.read_h5ad('/work/home/cryoem666/xyf/temp/pycharm/state/data/state_original_hf_hepg2/adata_pred.h5ad')
hepg2_real = ad.read_h5ad('/work/home/cryoem666/xyf/temp/pycharm/state/data/state_original_hf_hepg2/adata_real.h5ad')

In [11]:
hepg2_real.obs['gene'].unique()

['RPL35', 'HSD17B10', 'GPS1', 'NUP88', 'GCLC', ..., 'ZNF787', 'GTF2H3', 'SETX', 'SDHC', 'VARS2']
Length: 381
Categories (381, object): ['ABCB7', 'ABHD11', 'ACTR1B', 'ACTR8', ..., 'ZNF787', 'ZNHIT3', 'ZNHIT6', 'non-targeting']

In [12]:
state_Data_train = state_Data[~((state_Data.obs['cell_line']=='hepg2')&(state_Data.obs['gene'].isin(test_gene_list)))] 

In [13]:
state_Data_train[(state_Data_train.obs['cell_line']=='hepg2')&(state_Data_train.obs['gene']=='non-targeting')]

View of AnnData object with n_obs × n_vars = 4976 × 6642
    obs: 'gem_group', 'gene', 'gene_id', 'transcript', 'gene_transcript', 'sgID_AB', 'mitopercent', 'UMI_count', 'z_gemgroup_UMI', 'cell_line', 'perturb_label', 'batch_label', 'cell_line_label'
    var: 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    uns: 'hvg'
    obsm: 'X_hvg'

In [168]:
direction_dict = {}
direction_total = {}
cell_type_list = ['k562', 'jurkat', 'rpe1', 'hepg2']
#gene_list = ['CDC20']
gene_list = hepg2_real.obs['pert_name'].unique()
import numpy as np
import torch
from tqdm import tqdm
for gene in tqdm(gene_list):
    for cell_type in cell_type_list:
        if cell_type == 'hepg2' and gene!='non-targeting' :
            continue
        key = (gene, cell_type)
        case = state_Data_train[(state_Data_train.obs['gene']==gene) & (state_Data_train.obs['cell_line']==cell_type)].copy()
        if case.n_obs == 0:
            continue
        case.obsm['VAE_latent'] = sampler.autoencoder(torch.tensor(case[:, case.var['highly_variable']].X.toarray()).to('cuda'), return_latent=True).detach().cpu().numpy()
        z_sem = sampler.model.encoder.forward(
            torch.tensor(case.obsm['VAE_latent']).to('cuda'),
            cell_type = torch.tensor(np.stack(case.obs['cell_line_label'].values), dtype=torch.int32).to('cuda'),
            batch = torch.tensor(np.stack(case.obs['batch_label'].values), dtype=torch.int32).to('cuda'),
            perturb_label = torch.tensor(np.stack(case.obs['perturb_label'].values), dtype=torch.int32).to('cuda'),
        )
        z_sem_mean = z_sem.mean(axis=0).detach().cpu().numpy()
        direction_dict[key] = z_sem_mean
        direction_total[key] = z_sem

100%|██████████| 381/381 [00:21<00:00, 17.89it/s]


In [16]:
avg_direction = {}
for gene in hepg2_real.obs['gene'].unique():
    if gene == 'non-targeting':
        continue
    control2perturb = [value-direction_dict[('non-targeting', key[1])] for key, value in direction_dict.items() if key[0]==gene]
    avg_direction[gene] = np.mean(control2perturb, axis=0)

/work/home/cryoem666/miniconda3/envs/my_state/lib/python3.12/site-packages/numpy/_core/fromnumeric.py:3860: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/work/home/cryoem666/miniconda3/envs/my_state/lib/python3.12/site-packages/numpy/_core/_methods.py:144: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [17]:

direction_dict[('RPL35','k562')], direction_dict[('RPL35','rpe1')]
#Compute cosine similarity between two vectors
def cosine_similarity(vec1, vec2):
    dot_product = np.dot(vec1, vec2)
    norm_vec1 = np.linalg.norm(vec1)
    norm_vec2 = np.linalg.norm(vec2)
    if norm_vec1 == 0 or norm_vec2 == 0:
        return 0.0
    return dot_product / (norm_vec1 * norm_vec2)
cosine_similarity(direction_dict[('RPL35','k562')], direction_dict[('RPL35','rpe1')])

np.float32(0.07984416)

In [37]:
import torch

@torch.no_grad()
def _median_heuristic(D2):
    # D2: [N, N] squared distances
    # 去掉对角线 0，取中位数更稳
    N = D2.shape[0]
    mask = ~torch.eye(N, dtype=torch.bool, device=D2.device)
    return torch.median(D2[mask])

def mmd_loss(source, target, kernel_mul=2.0, kernel_num=5, fix_sigma=None, device="cuda"):
    source = torch.as_tensor(source, device=device, dtype=torch.float32)
    target = torch.as_tensor(target, device=device, dtype=torch.float32)
    n, m = source.size(0), target.size(0)
    total = torch.cat([source, target], dim=0)  # [N, d]
    N = n + m

    # [N, N] 欧氏距离；cdist返回的是sqrt的距离，这里平方
    D2 = torch.cdist(total, total, p=2) ** 2

    if fix_sigma is None:
        # 更常用的 median heuristic
        sigma2 = _median_heuristic(D2).clamp_min(1e-12)
    else:
        sigma2 = torch.tensor(fix_sigma, device=device, dtype=torch.float32)

    # 多核：sigma2 / kernel_mul^(k//2) 作为基准
    base = sigma2 / (kernel_mul ** (kernel_num // 2))
    K = 0.0
    for i in range(kernel_num):
        gamma = base * (kernel_mul ** i)
        K = K + torch.exp(-D2 / (2.0 * gamma))

    XX = K[:n, :n]
    YY = K[n:, n:]
    XY = K[:n, n:]
    YX = K[n:, :n]
    return XX.mean() + YY.mean() - XY.mean() - YX.mean()

import torch
import torch.nn.functional as F

def _spearman_corr_1d(x, y, eps=1e-8):
    """
    x, y: 1D tensor, same length
    返回 Spearman 相关系数（标量）
    """
    x = x.float()
    y = y.float()

    # rank: argsort().argsort() 简单实现，不做复杂 ties 校正
    rx = x.argsort().argsort().float()
    ry = y.argsort().argsort().float()

    rx = rx - rx.mean()
    ry = ry - ry.mean()
    rx = rx / (rx.std(unbiased=False) + eps)
    ry = ry / (ry.std(unbiased=False) + eps)

    return (rx * ry).mean()


def scc_score(source, target, device="cuda"):
    """
    source, target: [..., n_genes]，默认第一维是 cell
    建议外面先做好 log-normalization，再丢进来，
    对齐 scDiffusion 那种做法。
    """
    source = torch.as_tensor(source, device=device, dtype=torch.float32)
    target = torch.as_tensor(target, device=device, dtype=torch.float32)

    # [n_genes]，在 cell 维取均值
    mu_s = source.mean(dim=0)
    mu_t = target.mean(dim=0)

    return _spearman_corr_1d(mu_s, mu_t)
def lisi_binary(source, target, k=90, device="cuda"):
    """
    kNN 版 iLISI：
    - source: [n, d]
    - target: [m, d]
    - 返回整体平均 LISI，以及 source / target 各自的平均 LISI
    """
    source = torch.as_tensor(source, device=device, dtype=torch.float32)
    target = torch.as_tensor(target, device=device, dtype=torch.float32)

    n, d = source.shape
    m, _ = target.shape

    total = torch.cat([source, target], dim=0)  # [N, d]
    N = n + m

    labels = torch.cat([
        torch.zeros(n, device=device, dtype=torch.long),
        torch.ones(m, device=device, dtype=torch.long)
    ], dim=0)  # 0 = source, 1 = target

    # 跟你 MMD 一样，直接用 cdist
    D2 = torch.cdist(total, total, p=2) ** 2  # [N, N]

    # kNN：取自己之外的最近 k 个
    # indices[:, 0] 是自己
    knn_idx = D2.topk(k + 1, largest=False).indices[:, 1:]  # [N, k]

    knn_labels = labels[knn_idx]  # [N, k]

    # one-hot 统计邻居标签分布
    num_classes = int(labels.max().item()) + 1  # 应该是 2
    one_hot = F.one_hot(knn_labels, num_classes=num_classes).float()  # [N, k, C]

    # p_il = 邻居中各 label 的比例
    p = one_hot.mean(dim=1)  # [N, C]

    # LISI_i = 1 / sum_l p_il^2
    lisi_i = 1.0 / (p ** 2).sum(dim=1)  # [N]

    # 总体 & 各子集平均
    lisi_all = lisi_i.mean()
    lisi_source = lisi_i[:n].mean()
    lisi_target = lisi_i[n:].mean()

    return lisi_all, lisi_source, lisi_target

def return_metrics(x, y):
    from sklearn.metrics import r2_score
    from scipy.stats import pearsonr
    return {"r2_score": r2_score(x.mean(axis=0), y.mean(axis=0)), 
            "pearsonr": pearsonr(x.mean(axis=0), y.mean(axis=0))[0], 
            "mmd": mmd_loss(x, y), 
            "mmd_rbf": mmd_rbf(x,y)}

In [ ]:
def guassian_kernel(source, target, kernel_mul=2.0, kernel_num=5, fix_sigma=None):
    n_samples = int(source.size()[0])+int(target.size()[0])
    total = torch.cat([source, target], dim=0)

    total0 = total.unsqueeze(0).expand(int(total.size(0)), int(total.size(0)), int(total.size(1)))
    total1 = total.unsqueeze(1).expand(int(total.size(0)), int(total.size(0)), int(total.size(1)))
    
    batch_size = 200
    num_window = int(total0.shape[0]/batch_size)+1
    L2_dis = []
    for i in tqdm(range(num_window)):
        diff = (total0[i*batch_size:(i+1)*batch_size].cuda()-total1[i*batch_size:(i+1)*batch_size].cuda())
        diff.square_()
        L2_dis.append(diff.sum(2).cpu())
    L2_distance = torch.concatenate(L2_dis,dim=0)


    if fix_sigma:
        bandwidth = fix_sigma
    else:
        bandwidth = torch.sum(L2_distance.data) / (n_samples**2-n_samples)

    bandwidth /= kernel_mul ** (kernel_num // 2)
    bandwidth_list = [bandwidth * (kernel_mul**i) for i in range(kernel_num)]

    kernel_val = [torch.exp(-L2_distance / bandwidth_temp) for bandwidth_temp in bandwidth_list]

    return sum(kernel_val)#/len(kernel_val)

def mmd_rbf(source, target, kernel_mul=2.0, kernel_num=5, fix_sigma=None):
    source = torch.as_tensor(source, dtype=torch.float32).cuda()
    target = torch.as_tensor(target, dtype=torch.float32).cuda()
    batch_size = int(source.size()[0])
    kernels = guassian_kernel(source, target,
        kernel_mul=kernel_mul, kernel_num=kernel_num, fix_sigma=fix_sigma)

    XX = kernels[:batch_size, :batch_size]
    YY = kernels[batch_size:, batch_size:]
    XY = kernels[:batch_size, batch_size:]
    YX = kernels[batch_size:, :batch_size]
    loss = torch.mean(XX + YY - XY -YX)
    return loss

In [27]:
case = state_Data_train[(state_Data_train.obs['cell_line']=='hepg2')&(state_Data_train.obs['gene']=='non-targeting')]
hepg2_pred = ad.read_h5ad('/work/home/cryoem666/xyf/temp/pycharm/state-reproduce/baselines/baseline_output/cpa_replogle_v2/fold1/adata_pred.h5ad')
hepg2_real = ad.read_h5ad('/work/home/cryoem666/xyf/temp/pycharm/state-reproduce/baselines/baseline_output/cpa_replogle_v2/fold1/adata_real.h5ad')

In [ ]:
import random
pert = random.choice(hepg2_real.obs['pert_name'].unique())
#pert = 'non-targeting'
hepg2_real_sample = hepg2_real[hepg2_real.obs['pert_name']==pert].copy()
hepg2_pred_sample = hepg2_pred[hepg2_pred.obs['pert_name']==pert].copy()
import numpy as np
import torch
from tqdm import tqdm

# 1) 预分配 obsm
hepg2_real_sample.obsm['X_squidiff_ours'] = np.zeros((hepg2_real_sample.n_obs, 2000), dtype=np.float32)

# 2) 准备常量（放 GPU 一次就好）
base_control = torch.as_tensor(
    direction_dict[('non-targeting','hepg2')],
    device='cuda',
    dtype=torch.float32
)  # [128]

for gene in tqdm(hepg2_real_sample.obs['pert_name'].unique()):
    mask = (hepg2_real_sample.obs['pert_name'].values == gene)
    n = int(mask.sum())
    if n == 0:
        continue
    
    if gene == 'non-targeting':
        scrna_pred = hepg2_real_sample.X[mask]
        hepg2_real_sample.obsm['X_squidiff_ours'][mask] = scrna_pred
        continue
    '''
    if gene == 'non-targeting':
        new_direction = torch.zeros(60, device='cuda', dtype=torch.float32)
    '''    
    new_direction = torch.as_tensor(avg_direction[gene], device='cuda', dtype=torch.float32)  # [128]
    # [n, 128]
    z_sem_origin = base_control.unsqueeze(0).expand(n, -1).contiguous()

    with torch.no_grad():
        scrna_pred = sampler.interp_with_direction(
            z_sem_origin=z_sem_origin,
            gene_size=128,
            direction=new_direction,
            scale=1,
            add_noise_term=False
        )
        '''
        scrna_pred_new = sampler.pred(
            z_sem = z_sem_origin,
            gene_size = 128
        )
        '''

    hepg2_real_sample.obsm['X_squidiff_ours'][mask] = scrna_pred


  0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:02<00:00,  2.88s/it]


In [41]:
non_targeting_hepg2 = state_Data_train[(state_Data_train.obs['cell_line']=='hepg2')&(state_Data_train.obs['gene']=='non-targeting')]
non_targeting_hepg2.obsm['VAE_latent'] = sampler.autoencoder(torch.tensor(non_targeting_hepg2[:, non_targeting_hepg2.var['highly_variable']].X.toarray()).to('cuda'), return_latent=True).detach().cpu().numpy()
z_sem = sampler.model.encoder(
    torch.tensor(non_targeting_hepg2.obsm['VAE_latent']).to('cuda'),
    cell_type = torch.tensor(np.stack(non_targeting_hepg2.obs['cell_line_label'].values), dtype=torch.int32).to('cuda'),
    batch = torch.tensor(np.stack(non_targeting_hepg2.obs['batch_label'].values), dtype=torch.int32).to('cuda'),
    perturb_label = torch.tensor(np.stack(non_targeting_hepg2.obs['perturb_label'].values), dtype=torch.int32).to('cuda')
)
pred = sampler.pred(
    z_sem = z_sem,
    gene_size = 128
)

/tmp/ipykernel_211389/1963521033.py:2: ImplicitModificationWarning: Setting element `.obsm['VAE_latent']` of view, initializing view as actual.
  non_targeting_hepg2.obsm['VAE_latent'] = sampler.autoencoder(torch.tensor(non_targeting_hepg2[:, non_targeting_hepg2.var['highly_variable']].X.toarray()).to('cuda'), return_latent=True).detach().cpu().numpy()


In [44]:
non_targeting_hepg2

AnnData object with n_obs × n_vars = 4976 × 6642
    obs: 'gem_group', 'gene', 'gene_id', 'transcript', 'gene_transcript', 'sgID_AB', 'mitopercent', 'UMI_count', 'z_gemgroup_UMI', 'cell_line', 'perturb_label', 'batch_label', 'cell_line_label'
    var: 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    uns: 'hvg'
    obsm: 'X_hvg', 'VAE_latent'

In [116]:
return_metrics(hepg2_real_sample.X, hepg2_real_sample.obsm['X_squidiff_ours'])

100%|██████████| 1/1 [00:00<00:00, 824.68it/s]


{'r2_score': 0.9749298095703125,
 'pearsonr': np.float32(0.9881098),
 'mmd': tensor(0.9742, device='cuda:0'),
 'mmd_rbf': tensor(1.4266)}

In [169]:
import numpy as np
import torch
from tqdm import tqdm

# 1) 预分配 obsm
hepg2_real.obsm['X_squidiff_ours'] = np.zeros((hepg2_real.n_obs, 2000), dtype=np.float32)

# 2) 准备常量（放 GPU 一次就好）
base_control = torch.as_tensor(
    direction_dict[('non-targeting','hepg2')],
    device='cuda',
    dtype=torch.float32
)  # [128]

for gene in tqdm(hepg2_real.obs['pert_name'].unique()):
    mask = (hepg2_real.obs['pert_name'].values == gene)
    n = int(mask.sum())
    if n == 0:
        continue
    if gene == 'non-targeting':
        scrna_pred = hepg2_real.X[mask]
        hepg2_real.obsm['X_squidiff_ours'][mask] = scrna_pred
        continue
    new_direction = torch.as_tensor(avg_direction[gene], device='cuda', dtype=torch.float32)  # [128]

    # [n, 128]
    z_sem_origin = base_control.unsqueeze(0).expand(n, -1).contiguous()

    with torch.no_grad():
        scrna_pred = sampler.interp_with_direction(
            z_sem_origin=z_sem_origin,
            gene_size=128,
            direction=new_direction,
            scale=1
        )

    scrna_pred = scrna_pred
    hepg2_real.obsm['X_squidiff_ours'][mask] = scrna_pred


100%|██████████| 381/381 [16:41<00:00,  2.63s/it]


In [176]:
import numpy as np
import torch
from tqdm import tqdm

# 1) 预分配 obsm
hepg2_real.obsm['X_squidiff_ours'] = np.zeros((hepg2_real.n_obs, 2000), dtype=np.float32)

# 2) 准备常量（放 GPU 一次就好）
base_control = torch.as_tensor(
    direction_dict[('non-targeting','hepg2')],
    device='cuda',
    dtype=torch.float32
)  # [128]

for gene in tqdm(hepg2_real.obs['pert_name'].unique()):
    mask = (hepg2_real.obs['pert_name'].values == gene)
    n = int(mask.sum())
    if n == 0:
        continue
    if gene == 'non-targeting':
        scrna_pred = hepg2_real.X[mask]
        hepg2_real.obsm['X_squidiff_ours'][mask] = scrna_pred
        continue
    new_direction = torch.as_tensor(avg_direction[gene], device='cuda', dtype=torch.float32)  # [128]
    # [n, 128]
    z_sem_origin = base_control.unsqueeze(0).expand(n, -1).contiguous()
    with torch.no_grad():
        if isinstance(np.isnan(avg_direction[gene]),np.bool_):
            scrna_pred = sampler.pred(
                z_sem = z_sem_origin,
                gene_size = 128
            )
        else:
            scrna_pred = sampler.interp_with_direction(
                z_sem_origin=z_sem_origin,
                gene_size=128,
                direction=new_direction,
                scale=0.5,
                add_noise_term = False
            )

    scrna_pred = scrna_pred
    hepg2_real.obsm['X_squidiff_ours'][mask] = scrna_pred


100%|██████████| 381/381 [16:34<00:00,  2.61s/it]


array([1.3019072, 0.8493603, 0.       , ..., 4.7882867, 3.415735 ,
       0.8493603], shape=(2000,), dtype=float32)

In [146]:
isinstance(np.isnan(avg_direction['CDC20']),np.bool_)

True

In [170]:
[item for item in list(hepg2_real.obs['pert_name'].unique()) if item!='non-targeting' and isinstance(np.isnan(avg_direction[item]),np.bool_)]

['CDC20', 'NOS1AP', 'SLC16A5', 'CMTR2', 'CACTIN', 'SPAG7', 'MRM1', 'LAMB1']

In [156]:
nan_list = [item for item in list(hepg2_real.obs['pert_name'].unique()) if item!='non-targeting' and isinstance(np.isnan(avg_direction[item]),np.bool_)]
for gene in nan_list:
    hepg2_new.X[hepg2_new.obs['pert_name']==gene] = hepg2_real.X[hepg2_real.obs['pert_name']==gene]

In [177]:
hepg2_new = hepg2_real.copy()
hepg2_new.X = hepg2_new.obsm['X_squidiff_ours']
hepg2_new.obsm = {}
hepg2_real.obsm = {}

In [178]:
nan_count = np.isnan(hepg2_new.X).sum()
nan_count

np.int64(0)

In [97]:
hepg2_new.X[12:].shape

(31842, 2000)

In [162]:
hepg2_real.X

array([[1.7051982 , 0.91662085, 0.        , ..., 5.165333  , 2.94496   ,
        0.91662085],
       [0.96521634, 0.        , 0.        , ..., 4.6695347 , 3.2960477 ,
        1.622036  ],
       [2.0225873 , 0.        , 0.        , ..., 4.669593  , 2.5942886 ,
        0.8990636 ],
       ...,
       [0.904325  , 1.1648356 , 0.        , ..., 5.157652  , 2.6024854 ,
        0.55108345],
       [1.0081192 , 0.        , 0.        , ..., 4.7771883 , 2.3853168 ,
        1.2001423 ],
       [1.2436713 , 0.        , 0.        , ..., 4.704373  , 2.9497426 ,
        1.4565426 ]], shape=(31854, 2000), dtype=float32)

In [163]:
hepg2_new.X

array([[1.7051982 , 0.91662085, 0.        , ..., 5.165333  , 2.94496   ,
        0.91662085],
       [0.96521634, 0.        , 0.        , ..., 4.6695347 , 3.2960477 ,
        1.622036  ],
       [2.0225873 , 0.        , 0.        , ..., 4.669593  , 2.5942886 ,
        0.8990636 ],
       ...,
       [0.904325  , 1.1648356 , 0.        , ..., 5.157652  , 2.6024854 ,
        0.55108345],
       [1.0081192 , 0.        , 0.        , ..., 4.7771883 , 2.3853168 ,
        1.2001423 ],
       [1.2436713 , 0.        , 0.        , ..., 4.704373  , 2.9497426 ,
        1.4565426 ]], shape=(31854, 2000), dtype=float32)

In [180]:
import numpy as np

# 检查负值情况
neg_mask = hepg2_new.X < 0
neg_count = neg_mask.sum()
total_elements = hepg2_new.X.size

print(f"Negative values: {neg_count}/{total_elements} ({neg_count/total_elements*100:.2f}%)")
print(f"Min value: {hepg2_new.X.min():.6f}")
print(f"Max value: {hepg2_new.X.max():.6f}")

# 方法1：直接截断负值为0（最常用）
hepg2_new.X = np.maximum(hepg2_new.X, 0)

print(f"After truncation:")
print(f"Min value: {hepg2_new.X.min():.6f}")
print(f"Max value: {hepg2_new.X.max():.6f}")
hepg2_real.write_h5ad('/work/home/cryoem666/xyf/temp/pycharm/Squidiff/Squidiff_reproducibility/data/hepg2_squidiff_ours_full_scale0.5_real.h5ad')   
hepg2_new.write_h5ad('/work/home/cryoem666/xyf/temp/pycharm/Squidiff/Squidiff_reproducibility/data/hepg2_squidiff_ours_full_scale0.5_pred.h5ad')   

Negative values: 0/63708000 (0.00%)
Min value: 0.000000
Max value: 6.251856
After truncation:
Min value: 0.000000
Max value: 6.251856


In [181]:
hepg2_real.X, hepg2_new.X

(array([[1.7051982 , 0.91662085, 0.        , ..., 5.165333  , 2.94496   ,
         0.91662085],
        [0.96521634, 0.        , 0.        , ..., 4.6695347 , 3.2960477 ,
         1.622036  ],
        [2.0225873 , 0.        , 0.        , ..., 4.669593  , 2.5942886 ,
         0.8990636 ],
        ...,
        [0.904325  , 1.1648356 , 0.        , ..., 5.157652  , 2.6024854 ,
         0.55108345],
        [1.0081192 , 0.        , 0.        , ..., 4.7771883 , 2.3853168 ,
         1.2001423 ],
        [1.2436713 , 0.        , 0.        , ..., 4.704373  , 2.9497426 ,
         1.4565426 ]], shape=(31854, 2000), dtype=float32),
 array([[1.0479224 , 0.58396006, 0.16260432, ..., 4.8067646 , 2.9952588 ,
         1.0597457 ],
        [0.3928091 , 0.5001398 , 0.13378525, ..., 4.699545  , 3.0342236 ,
         1.0674665 ],
        [0.44279248, 0.40502602, 0.14908704, ..., 4.6760817 , 2.7671125 ,
         1.0901204 ],
        ...,
        [0.7499684 , 0.51036096, 0.15180391, ..., 4.713362  , 3.0377276 

In [85]:
hepg2_pred.X , hepg2_real.X = hepg2_pred.X.astype(np.float32), hepg2_real.X.astype(np.float32)

In [ ]:
from tqdm import tqdm 
control = [direction_dict[('non-targeting','hepg2')]]
total_pred = []
for  in tqdm(range(hepg2_real.n_obs)): 
    x_i, obs_i = hepg2_real.X[i], hepg2_real.obs.iloc[i]
    perturb, cell_type = obs_i['gene'], obs_i['cell_type']
    new_direction = avg_direction[perturb]
    if perturb == 'non-targeting':
        scrna_pred = x_i
        total_pred.append(scrna_pred)
        continue
    scrna_pred = sampler.interp_with_direction(
        z_sem_origin = torch.tensor(control).to('cuda'),
        gene_size = 128,
        direction = torch.tensor(new_direction).to('cuda'),
        scale = 1
    )
    total_pred.append(scrna_pred)

  0%|          | 0/28337 [00:00<?, ?it/s]/tmp/ipykernel_177920/659472469.py:13: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /pytorch/torch/csrc/utils/tensor_new.cpp:254.)
  z_sem_origin = torch.tensor(control).to('cuda'),
  0%|          | 8/28337 [00:23<23:15:08,  2.95s/it]


KeyboardInterrupt: 

In [ ]:
import numpy as np
hepg2_pred_squidiff_ours = hepg2_real.copy()
hepg2_pred_squidiff_ours.X = np.array(total_pred)

In [109]:
import torch

case = state_Data[(state_Data.obs['gene']==perturb_list[0]) & (state_Data.obs['cell_line']=='hepg2')]
case.obsm['VAE_latent'] = sampler.autoencoder(torch.tensor(case[:, case.var['highly_variable']].X.toarray()).to('cuda'), return_latent=True).detach().cpu().numpy()
z_sem = sampler.model.encoder.forward(
    torch.tensor(case.obsm['VAE_latent']).to('cuda'),
    cell_type = torch.tensor(np.stack(case.obs['cell_line_label'].values), dtype=torch.int32).to('cuda'),
    batch = torch.tensor(np.stack(case.obs['batch_label'].values), dtype=torch.int32).to('cuda'),
    perturb_label = torch.tensor(np.stack(case.obs['perturb_label'].values), dtype=torch.int32).to('cuda'),
)

scrnas_pred = sampler.pred(z_sem, 128)
     

/tmp/ipykernel_123690/983445336.py:4: ImplicitModificationWarning: Setting element `.obsm['VAE_latent']` of view, initializing view as actual.
  case.obsm['VAE_latent'] = sampler.autoencoder(torch.tensor(case[:, case.var['highly_variable']].X.toarray()).to('cuda'), return_latent=True).detach().cpu().numpy()


In [110]:
x, y = case[:, case.var['highly_variable']].X, scrnas_pred

In [61]:
hepg2_pred = ad.read_h5ad('/work/home/cryoem666/xyf/temp/pycharm/state/data/state_original_hf_hepg2/adata_pred.h5ad')
hepg2_real = ad.read_h5ad('/work/home/cryoem666/xyf/temp/pycharm/state/data/state_original_hf_hepg2/adata_real.h5ad')

In [72]:
hepg2_pred = ad.read_h5ad('/work/home/cryoem666/xyf/temp/pycharm/state-reproduce/baselines/baseline_output/cpa_replogle_v2/fold1/adata_pred.h5ad')
hepg2_real = ad.read_h5ad('/work/home/cryoem666/xyf/temp/pycharm/state-reproduce/baselines/baseline_output/cpa_replogle_v2/fold1/adata_real.h5ad')

In [106]:
lrlm_pred = ad.read_h5ad('/work/home/cryoem666/xyf/temp/pycharm/state-reproduce/baselines/baseline_output/lrlm_replogle_v2/fold1/adata_pred.h5ad')
lrlm_real = ad.read_h5ad('/work/home/cryoem666/xyf/temp/pycharm/state-reproduce/baselines/baseline_output/lrlm_replogle_v2/fold1/adata_real.h5ad')

In [ ]:
mmd_loss

NameError: name 'hepg2_pred' is not defined

In [107]:
mmd_dict_lrlm = {}
for perturb in perturb_list:
    mmd_dict_lrlm[perturb] = mmd_loss(lrlm_real[scvi_real.obs['pert_name']==perturb].X.toarray(), lrlm_pred[lrlm_pred.obs['pert_name']==perturb].X.toarray())

IndexError: Boolean index does not match AnnData’s shape along this dimension. Boolean index has shape (31854,) while AnnData index has shape (31545,).

In [98]:
mmd_dict_scvi = {}
for perturb in perturb_list:
    mmd_dict_scvi[perturb] = mmd_loss(scvi_real[scvi_real.obs['pert_name']==perturb].X.toarray(), scvi_pred[scvi_pred.obs['pert_name']==perturb].X.toarray())

In [102]:
mmd_dict_scvi.values()

dict_values([tensor(1.1719, device='cuda:0'), tensor(1.2397, device='cuda:0'), tensor(1.1240, device='cuda:0'), tensor(1.1201, device='cuda:0'), tensor(1.1806, device='cuda:0'), tensor(1.2235, device='cuda:0'), tensor(1.2302, device='cuda:0'), tensor(1.2562, device='cuda:0'), tensor(1.2109, device='cuda:0'), tensor(1.2344, device='cuda:0'), tensor(1.3507, device='cuda:0'), tensor(1.1334, device='cuda:0'), tensor(1.2518, device='cuda:0'), tensor(1.2789, device='cuda:0'), tensor(1.2349, device='cuda:0'), tensor(1.1931, device='cuda:0'), tensor(1.1786, device='cuda:0'), tensor(1.1908, device='cuda:0'), tensor(1.2136, device='cuda:0'), tensor(1.2522, device='cuda:0'), tensor(1.2078, device='cuda:0'), tensor(1.2285, device='cuda:0'), tensor(1.2760, device='cuda:0'), tensor(1.1847, device='cuda:0'), tensor(1.1848, device='cuda:0'), tensor(1.1619, device='cuda:0'), tensor(1.1951, device='cuda:0'), tensor(1.2461, device='cuda:0'), tensor(1.1277, device='cuda:0'), tensor(1.2273, device='cuda:0'

In [104]:
mmd_dict = {}
for perturb in perturb_list:
    mmd_dict[perturb] = mmd_loss(hepg2_real[hepg2_real.obs['gene']==perturb].X.toarray(), hepg2_pred[hepg2_pred.obs['gene']==perturb].X.toarray())

In [103]:
mmd_dict

{'UROD': tensor(1.1719, device='cuda:0'),
 'GFER': tensor(1.2397, device='cuda:0'),
 'MED8': tensor(1.1240, device='cuda:0'),
 'C9orf16': tensor(1.1201, device='cuda:0'),
 'HNRNPDL': tensor(1.1806, device='cuda:0'),
 'CCP110': tensor(1.2235, device='cuda:0'),
 'TRMT112': tensor(1.2302, device='cuda:0'),
 'TSFM': tensor(1.2562, device='cuda:0'),
 'NUDC': tensor(1.2109, device='cuda:0'),
 'SMNDC1': tensor(1.2344, device='cuda:0'),
 'AGBL5': tensor(1.3507, device='cuda:0'),
 'CCDC6': tensor(1.1334, device='cuda:0'),
 'SOD2': tensor(1.2518, device='cuda:0'),
 'NELFE': tensor(1.2789, device='cuda:0'),
 'COG2': tensor(1.2349, device='cuda:0'),
 'CYFIP1': tensor(1.1931, device='cuda:0'),
 'SDHC': tensor(1.1786, device='cuda:0'),
 'C19orf53': tensor(1.1908, device='cuda:0'),
 'MSL1': tensor(1.2136, device='cuda:0'),
 'PPIL2': tensor(1.2522, device='cuda:0'),
 'CLPB': tensor(1.2078, device='cuda:0'),
 'BCLAF1': tensor(1.2285, device='cuda:0'),
 'COPS2': tensor(1.2760, device='cuda:0'),
 'RPP14'

In [78]:
perturb_list = list(set(hepg2_real.obs['gene']))

In [67]:
hepg2_pred[hepg2_pred.obs['gene']=='non-targeting']

View of AnnData object with n_obs × n_vars = 1459 × 2000
    obs: 'gene', 'cell_type', 'ct_gem_group', 'pert_cell_barcode', 'ctrl_cell_barcode'

In [120]:
from scipy.stats import pearsonr
from sklearn.metrics import r2_score
#x, y = scrnas_pred[0].mean(axis=0).detach().cpu().numpy(),case.obsm['VAE_latent'].mean(axis=0)
def return_metrics(x, y):
    return r2_score(x.mean(axis=0), y.mean(axis=0)), pearsonr(x.mean(axis=0), y.mean(axis=0))[0], mmd_loss(x, y)

In [55]:
test_gene_list = config['fewshot']['replogle_proper.hepg2']['test']

In [56]:
len(set(state_Data[(state_Data.obs['gene'].isin(test_gene_list))&(state_Data.obs['cell_line']=='hepg2')].obs['gene']))

372

In [ ]:
from cell_eval import 

In [17]:
#拿ttf2作为例子
import torch
perturb, control_label =  "RPS8", "non-targeting"
perturb_state_data = state_Data[state_Data.obs['gene'] == perturb]
perturb_direction_dict = {}
perturb_profile_dict = {}
control_profile_dict = {}
z_perturb_dict = {}
z_control_dict = {}
from tqdm import tqdm 
for cell_type in tqdm(cell_type_list, position=0):
    print(f"Processing cell type: {cell_type}")
    celltype_perturb_Data = perturb_state_data[perturb_state_data.obs['cell_line'] == cell_type]
    control_Data = state_Data[(state_Data.obs['gene'] == control_label)&(state_Data.obs['cell_line'] == cell_type)]
    print(f"Cell type: {cell_type}, Perturbation data shape: {celltype_perturb_Data.shape}, Control data shape: {control_Data.shape}")
    if celltype_perturb_Data.shape[0] == 0 or control_Data.shape[0] == 0:
        print(f"Skipping cell type: {cell_type} due to insufficient data.")
        continue
    z_perturb = sampler.model.encoder.forward(
        x_start = torch.tensor(celltype_perturb_Data[:, state_Data.var['highly_variable']].X.toarray(), dtype=torch.float32).to('cuda'),
        perturb_label = torch.tensor(np.stack(celltype_perturb_Data.obs['perturb_label'].values), dtype=torch.int32).to('cuda'),
        batch = torch.tensor(np.stack(celltype_perturb_Data.obs['batch_label'].values), dtype=torch.int32).to('cuda'),
        cell_type = torch.tensor(np.stack(celltype_perturb_Data.obs['cell_line_label'].values), dtype=torch.int32).to('cuda')
    )
    print(f"Cell type: {cell_type}, Perturbation latent shape: {z_perturb.shape}")
    z_control = sampler.model.encoder.forward(
        x_start = torch.tensor(control_Data[:, state_Data.var['highly_variable']].X.toarray(), dtype=torch.float32).to('cuda'),
        perturb_label = torch.tensor(np.stack(control_Data.obs['perturb_label'].values), dtype=torch.int32).to('cuda'),
        batch = torch.tensor(np.stack(control_Data.obs['batch_label'].values), dtype=torch.int32).to('cuda'),
        cell_type = torch.tensor(np.stack(control_Data.obs['cell_line_label'].values), dtype=torch.int32).to('cuda')
    )
    print(f"Cell type: {cell_type}, Perturbation latent shape: {z_perturb.shape}, Control latent shape: {z_control.shape}")
    direction = z_perturb.mean(axis=0) - z_control.mean(axis=0)
    print(f"Cell type: {cell_type}, Direction shape: {direction.shape}")
    perturb_direction_dict[cell_type] = direction
    z_perturb_dict[cell_type] = z_perturb
    z_control_dict[cell_type] = z_control
    perturb_profile_dict[cell_type] = celltype_perturb_Data
    control_profile_dict[cell_type] = control_Data
    



  0%|          | 0/4 [00:00<?, ?it/s]

100%|██████████| 4/4 [00:00<00:00, 38.12it/s]

Processing cell type: jurkat
Cell type: jurkat, Perturbation data shape: (96, 6642), Control data shape: (200, 6642)
Cell type: jurkat, Perturbation latent shape: torch.Size([96, 512])
Cell type: jurkat, Perturbation latent shape: torch.Size([96, 512]), Control latent shape: torch.Size([200, 512])
Cell type: jurkat, Direction shape: torch.Size([512])
Processing cell type: k562
Cell type: k562, Perturbation data shape: (0, 6642), Control data shape: (200, 6642)
Skipping cell type: k562 due to insufficient data.
Processing cell type: rpe1
Cell type: rpe1, Perturbation data shape: (30, 6642), Control data shape: (200, 6642)
Cell type: rpe1, Perturbation latent shape: torch.Size([30, 512])
Cell type: rpe1, Perturbation latent shape: torch.Size([30, 512]), Control latent shape: torch.Size([200, 512])
Cell type: rpe1, Direction shape: torch.Size([512])
Processing cell type: hepg2
Cell type: hepg2, Perturbation data shape: (54, 6642), Control data shape: (200, 6642)
Cell type: hepg2, Perturba

In [18]:
z_perturb_dict['rpe1'].shape, z_perturb_dict['jurkat'].shape

(torch.Size([30, 512]), torch.Size([96, 512]))

In [19]:
# 看看latent在k562上的重建效果
sample_interp_k562_perturb = sampler.pred(
    z_sem = z_control_dict['rpe1'],
    gene_size = sum(state_Data.var['highly_variable'])
)
r2_, pearsonr_ = sampler.cal_metric(sample_interp_k562_perturb,control_profile_dict['rpe1'][:, state_Data.var['highly_variable']] )
# import MMD distance 


In [20]:
r2_, pearsonr_ 

(-16.654558181762695, 0.4131721)

In [22]:
x, y  = sample_interp_k562_perturb.to('cpu').detach().numpy(),control_profile_dict['rpe1'][:, state_Data.var['highly_variable']].X.toarray()

In [23]:
mmd_loss(x, y)

tensor(3.1213)

In [24]:
from sklearn.metrics import r2_score 
r2_score(
    control_profile_dict['k562'][:, state_Data.var['highly_variable']].X.toarray().flatten(),
    sample_interp_k562_perturb.cpu().numpy().flatten()
)

KeyError: 'k562'

In [28]:
import random 
test_data_gene1 = test_data[test_data.obs['gene']=='RPS8']
z_control = z_control_dict['hepg2'][random.sample(range(len(z_control_dict['hepg2'])), test_data_gene1.shape[0]),:]

In [29]:
sample_comb_list = {item: sampler.interp_with_direction(z_sem_origin = z_control_dict['hepg2'],
                                       gene_size = sum(state_Data.var['highly_variable']),
                                       direction = perturb_direction_dict[item],
                                       scale = 1,
                                       add_noise_term = False #add noise if for interpolation
                                      ) for item in cell_type_list if item in perturb_direction_dict.keys()}


In [30]:
from numpy import mean
from sklearn.metrics import r2_score
y = test_data_gene1[:, state_Data.var['highly_variable']].X.toarray()
for cell_type in sample_comb_list.keys():
    x = sample_comb_list[cell_type].to('cpu').detach().numpy()
    r2 = r2_score(
        y.mean(axis=0),
        x.mean(axis=0)
    )
    mmd = mmd_loss(x, y)
    print(f"Cell type: {cell_type}, R2 Score: {r2}, MMD Loss: {mmd.item()}")
for cell_type in sample_comb_list.keys():
    x = perturb_profile_dict[cell_type][:, state_Data.var['highly_variable']].X.toarray()
    r2 = r2_score(
        y.mean(axis=0),
        x.mean(axis=0)
    )
    mmd = mmd_loss(x, y)
    print(f"Cell type: {cell_type}, R2 Score: {r2}, MMD Loss: {mmd.item()}")
for cell_type in sample_comb_list.keys():
    x = control_profile_dict[cell_type][:, state_Data.var['highly_variable']].X.toarray()
    r2 = r2_score(
        y.mean(axis=0),
        x.mean(axis=0)
    )
    
    mmd = mmd_loss(x, y)
    print(f"Cell type: {cell_type}, R2 Score: {r2}, MMD Loss: {mmd.item()}")

Cell type: jurkat, R2 Score: -6927.0693359375, MMD Loss: 2.7360951900482178
Cell type: rpe1, R2 Score: -6878.76318359375, MMD Loss: 2.733095407485962
Cell type: hepg2, R2 Score: -6989.48486328125, MMD Loss: 2.7404024600982666
Cell type: jurkat, R2 Score: 0.4384259581565857, MMD Loss: 1.1788368225097656
Cell type: rpe1, R2 Score: 0.5646163821220398, MMD Loss: 1.0753662586212158
Cell type: hepg2, R2 Score: 1.0, MMD Loss: 0.0
Cell type: jurkat, R2 Score: 0.3381509780883789, MMD Loss: 1.3714734315872192
Cell type: rpe1, R2 Score: 0.4707518219947815, MMD Loss: 1.2071932554244995
Cell type: hepg2, R2 Score: 0.8574005365371704, MMD Loss: 0.4565865993499756


In [31]:
from numpy import mean
from sklearn.metrics import r2_score
y = control_profile_dict['hepg2'][:, state_Data.var['highly_variable']].X.toarray()
print("Compute distance between control data and generated data")
for cell_type in sample_comb_list.keys():
    x = sample_comb_list[cell_type].to('cpu').detach().numpy()
    r2 = r2_score(
        y.mean(axis=0),
        x.mean(axis=0)
    )
    mmd = mmd_loss(x, y)
    print(f"Cell type: {cell_type}, R2 Score: {r2}, MMD Loss: {mmd.item()}")
print("Compute distance between control data and actual perturbation data")
for cell_type in sample_comb_list.keys():
    x = perturb_profile_dict[cell_type][:, state_Data.var['highly_variable']].X.toarray()
    r2 = r2_score(
        y.mean(axis=0),
        x.mean(axis=0)
    )
    mmd = mmd_loss(x, y)
    print(f"Cell type: {cell_type}, R2 Score: {r2}, MMD Loss: {mmd.item()}")
print("Compute distance between control data and control data")
for cell_type in sample_comb_list.keys():
    x = control_profile_dict[cell_type][:, state_Data.var['highly_variable']].X.toarray()
    r2 = r2_score(
        y.mean(axis=0),
        x.mean(axis=0)
    )
    
    mmd = mmd_loss(x, y)
    print(f"Cell type: {cell_type}, R2 Score: {r2}, MMD Loss: {mmd.item()}")

Compute distance between control data and generated data
Cell type: jurkat, R2 Score: -6409.35205078125, MMD Loss: 3.118868112564087
Cell type: rpe1, R2 Score: -6364.76123046875, MMD Loss: 3.116450309753418
Cell type: hepg2, R2 Score: -6467.39892578125, MMD Loss: 3.122645854949951
Compute distance between control data and actual perturbation data
Cell type: jurkat, R2 Score: 0.5103132724761963, MMD Loss: 1.056046962738037
Cell type: rpe1, R2 Score: 0.5163160562515259, MMD Loss: 1.1418386697769165
Cell type: hepg2, R2 Score: 0.8679935932159424, MMD Loss: 0.45658671855926514
Compute distance between control data and control data
Cell type: jurkat, R2 Score: 0.5611333847045898, MMD Loss: 1.0134795904159546
Cell type: rpe1, R2 Score: 0.6404640674591064, MMD Loss: 0.9025882482528687
Cell type: hepg2, R2 Score: 1.0, MMD Loss: 0.0


In [262]:
pred = ad.read_h5ad('/work/home/cryoem666/xyf/temp/pycharm/state/test_replogle/hepg2_holdout/eval_last.ckpt/adata_pred.h5ad')
truth = ad.read_h5ad('/work/home/cryoem666/xyf/temp/pycharm/state/test_replogle/hepg2_holdout/eval_last.ckpt/adata_real.h5ad')

In [268]:
truth.X

array([[0.77058756, 0.        , 0.        , ..., 3.2787578 , 0.77058756,
        4.4716063 ],
       [0.        , 1.305768  , 0.5143976 , ..., 2.716365  , 2.1281052 ,
        4.0400352 ],
       [0.6739177 , 0.27811354, 0.49547172, ..., 2.4766753 , 1.0728899 ,
        3.9258983 ],
       ...,
       [0.45875692, 0.        , 0.45875692, ..., 2.850647  , 1.5024396 ,
        4.433684  ],
       [0.74850327, 0.74850327, 0.44270754, ..., 2.9924858 , 1.1717597 ,
        4.1920652 ],
       [1.0784274 , 0.6780468 , 1.0784274 , ..., 2.8616548 , 0.6780468 ,
        4.4472556 ]], dtype=float32)

In [ ]:
from cell_eval import MetricsEvaluator
evaluator = MetricsEvaluator(
    adata_pred=pred_ct,
    adata_real=real_ct,
    de_pred=None,
    de_real=None,
    control_pert="non-targeting",
    pert_col="target",
    de_method="wilcoxon",
    num_threads=-1,
    batch_size=100,
    outdir="./cell-eval-outdir",
    allow_discrete=False,
    prefix=None,
    skip_de=False,
)

In [ ]:
from cell_eval impor

tensor([[1.5518, 1.0223, 0.4321,  ..., 0.0127, 0.2057, 0.7264],
        [0.7055, 0.7062, 0.7564,  ..., 0.0593, 0.0000, 1.0425],
        [0.7644, 1.2055, 0.3125,  ..., 0.0699, 0.3330, 0.8380],
        ...,
        [0.9589, 1.4247, 0.5020,  ..., 0.0650, 0.0365, 0.5431],
        [0.1778, 1.5413, 0.0811,  ..., 0.2136, 0.0000, 0.7665],
        [0.8655, 1.1622, 0.5321,  ..., 0.0669, 0.0123, 0.7398]],
       device='cuda:0')

In [164]:
torch.tensor(perturb_profile_dict['k562'][:, perturb_profile_dict['k562'].var['highly_variable']].X.toarray())

tensor([[0.4090, 0.6985, 1.1057,  ..., 0.0000, 0.6985, 2.5747],
        [0.8084, 0.8084, 1.0530,  ..., 0.0000, 0.8084, 2.4489],
        [0.5808, 0.5808, 0.0000,  ..., 0.5808, 0.5808, 1.5969],
        ...,
        [0.5166, 0.0000, 0.8555,  ..., 0.0000, 0.5166, 2.2099],
        [0.0000, 0.7911, 0.0000,  ..., 0.0000, 1.2272, 1.2272],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 2.1731]])

In [ ]:
def cal_metric_new()

In [167]:
matrix_dict = {}
for item in cell_type_list: 
    for type in cell_type_list:
        perturb_profile = torch.tensor(perturb_profile_dict[item][:, perturb_profile_dict[item].var['highly_variable']].X.toarray())
        matrix_dict[(item, type)] = sampler.cal_metric(perturb_profile, control_profile_dict[type][:, control_profile_dict[type].var['highly_variable']])

In [174]:
cell_type = 'hepg2'
cell_type_data = state_Data[state_Data.obs['cell_line']==cell_type]
matrix_dict_perturb = {}
gene_list = set(cell_type_data.obs['gene'])
gene_data = torch.tensor(cell_type_data[cell_type_data.obs['gene']=='non-targeting'][:, cell_type_data.var['highly_variable']].X.toarray())
for type in gene_list: 
    matrix_dict_perturb[('non-targeting', type)] = sampler.cal_metric(gene_data, cell_type_data[cell_type_data.obs['gene']==type][:, cell_type_data.var['highly_variable']])

In [ ]:
matrix_dict_perturb

{('non-targeting', 'RPS8'): (0.8290762901306152, 0.91209257),
 ('non-targeting', 'CCT6A'): (0.894419252872467, 0.9497758),
 ('non-targeting', 'non-targeting'): (1.0, 1.0),
 ('non-targeting', 'HGS'): (0.905807375907898, 0.95416015),
 ('non-targeting', 'POGZ'): (0.9838405847549438, 0.9919827),
 ('non-targeting', 'TRAPPC4'): (0.9801661968231201, 0.9903984),
 ('non-targeting', 'RIOK1'): (0.9225167036056519, 0.96107495),
 ('non-targeting', 'ATP2A2'): (0.9802168011665344, 0.9907773),
 ('non-targeting', 'PTCD3'): (0.9760352969169617, 0.9884068),
 ('non-targeting', 'POLR2B'): (0.8579238653182983, 0.9283825)}

In [168]:
matrix_dict

{('jurkat', 'jurkat'): (0.9882532954216003, 0.99437463),
 ('jurkat', 'k562'): (0.6175300478935242, 0.8205467),
 ('jurkat', 'rpe1'): (0.33593934774398804, 0.74484843),
 ('jurkat', 'hepg2'): (0.45138877630233765, 0.73489785),
 ('k562', 'jurkat'): (0.5508226156234741, 0.8077812),
 ('k562', 'k562'): (0.980702817440033, 0.99034506),
 ('k562', 'rpe1'): (-0.05782675743103027, 0.6352488),
 ('k562', 'hepg2'): (0.4683115482330322, 0.7432175),
 ('rpe1', 'jurkat'): (0.5514518618583679, 0.7467724),
 ('rpe1', 'k562'): (0.3809424638748169, 0.644668),
 ('rpe1', 'rpe1'): (0.9944376945495605, 0.9972677),
 ('rpe1', 'hepg2'): (0.7057881951332092, 0.84981686),
 ('hepg2', 'jurkat'): (0.4251304268836975, 0.73420304),
 ('hepg2', 'k562'): (0.4744534492492676, 0.7268021),
 ('hepg2', 'rpe1'): (0.5653258562088013, 0.8596075),
 ('hepg2', 'hepg2'): (0.9839951992034912, 0.9919827)}

In [157]:
cell_data = state_Data[(state_Data.obs['gene']=='POGZ')&(state_Data.obs['cell_line']=='hepg2')]
sampler.cal_metric(sample_comb_list['k562'], cell_data[:, cell_data.var['highly_variable']] )

(0.9664002656936646, 0.9851414)

In [141]:
sampler.cal_metric(sample_comb, state_Data[:37, state_Data.var['highly_variable']] )

(0.969586193561554, 0.9848137)

In [143]:
state_Data[:37, state_Data.var['highly_variable']].obs


,gem_group,gene,gene_id,transcript,gene_transcript,sgID_AB,mitopercent,UMI_count,z_gemgroup_UMI,cell_line,perturb_label,batch_label,cell_line_label
cell_barcode,,,,,,,,,,,,,
TCCCACACAAGTCCAT-20,20,non-targeting,non-targeting,non-targeting,10806_non-targeting_non-targeting_non-targeting,non-targeting_00318|non-targeting_00877,0.078166,12998.0,-0.126799,hepg2,10,19,0
AAGAACATCTAGTGTG-16,16,non-targeting,non-targeting,non-targeting,11012_non-targeting_non-targeting_non-targeting,non-targeting_01721|non-targeting_01433,0.099047,12388.0,0.759928,hepg2,10,15,0
GGGTATTAGGAACATT-42,42,non-targeting,non-targeting,non-targeting,11025_non-targeting_non-targeting_non-targeting,non-targeting_01813|non-targeting_00122,0.092756,12258.0,-0.521161,hepg2,10,41,0
TTGACCCCACGAAGAC-52,52,non-targeting,non-targeting,non-targeting,10951_non-targeting_non-targeting_non-targeting,non-targeting_01273|non-targeting_03357,0.131519,14918.0,0.026687,hepg2,10,51,0
GAGGGATCACTCCGGA-42,42,non-targeting,non-targeting,non-targeting,11271_non-targeting_non-targeting_non-targeting,non-targeting_03467|non-targeting_01317,0.022323,17336.0,0.131101,hepg2,10,41,0
AGGATAATCCCTCTCC-18,18,non-targeting,non-targeting,non-targeting,11208_non-targeting_non-targeting_non-targeting,non-targeting_02977|non-targeting_01146,0.096092,13383.0,0.638283,hepg2,10,17,0
GTGTTAGAGCATCTTG-8,8,non-targeting,non-targeting,non-targeting,10771_non-targeting_non-targeting_non-targeting,non-targeting_00164|non-targeting_00374,0.099374,18677.0,-0.044485,hepg2,10,7,0
CCAATGACATTGACAC-50,50,non-targeting,non-targeting,non-targeting,10967_non-targeting_non-targeting_non-targeting,non-targeting_01376|non-targeting_00354,0.078363,19716.0,0.358782,hepg2,10,49,0
TATATCCTCATGCAGT-1,1,non-targeting,non-targeting,non-targeting,10869_non-targeting_non-targeting_non-targeting,non-targeting_00693|non-targeting_02043,0.113725,15344.0,0.279445,hepg2,10,0,0


In [136]:
sample_comb.shape

torch.Size([37, 201])

In [132]:
state_Data[:37, state_Data.var['highly_variable']]

View of AnnData object with n_obs × n_vars = 37 × 201
    obs: 'gem_group', 'gene', 'gene_id', 'transcript', 'gene_transcript', 'sgID_AB', 'mitopercent', 'UMI_count', 'z_gemgroup_UMI', 'cell_line', 'perturb_label', 'batch_label', 'cell_line_label'
    var: 'highly_variable'
    obsm: 'X_hvg'

In [131]:
test_data_gene1[:, state_Data.var['highly_variable']]

View of AnnData object with n_obs × n_vars = 37 × 201
    obs: 'gem_group', 'gene', 'gene_id', 'transcript', 'gene_transcript', 'sgID_AB', 'mitopercent', 'UMI_count', 'z_gemgroup_UMI', 'cell_line', 'perturb_label', 'batch_label', 'cell_line_label'
    var: 'highly_variable'
    obsm: 'X_hvg'

In [130]:
sample_comb

tensor([[0.7282, 0.9856, 0.6788,  ..., 0.3601, 0.0000, 1.2772],
        [0.1632, 0.7973, 0.2425,  ..., 0.1355, 0.4455, 1.0979],
        [0.2565, 1.2333, 0.6216,  ..., 0.0000, 0.0631, 1.1449],
        ...,
        [1.0260, 1.0914, 0.3689,  ..., 0.0433, 0.0894, 1.1866],
        [0.6659, 1.0609, 0.2292,  ..., 0.0139, 0.2113, 1.1310],
        [0.3912, 1.1426, 0.3057,  ..., 0.2335, 0.0000, 1.1373]],
       device='cuda:0')

In [ ]:
gene_list = ['POGZ', 'RPS8']
for gene in gene_list:
    

In [105]:
#cosine similarity between different cell types
from sklearn.metrics.pairwise import cosine_similarity
cell_types = list(perturb_direction_dict.keys())
directions = torch.stack([perturb_direction_dict[ct] for ct in cell_types]).detach().cpu().numpy()
similarity_matrix = cosine_similarity(directions)

In [106]:
similarity_matrix

array([[ 1.0000001 ,  0.18834874, -0.03933998,  0.24443385],
       [ 0.18834874,  1.0000002 , -0.00648843,  0.01074759],
       [-0.03933998, -0.00648843,  1.        ,  0.09405403],
       [ 0.24443385,  0.01074759,  0.09405403,  0.99999994]],
      dtype=float32)

In [55]:
similarity_matrix

array([[ 0.99999994,  0.4493432 , -0.54073685,  0.3392582 ],
       [ 0.4493432 ,  0.99999976, -0.3435316 ,  0.12021162],
       [-0.54073685, -0.3435316 ,  1.0000001 , -0.64503545],
       [ 0.3392582 ,  0.12021162, -0.64503545,  1.0000001 ]],
      dtype=float32)

In [ ]:
test_gene = 

In [ ]:
test_ct = tuple(config['fewshot'].keys())[0].split('.')[1]
control_label = 'non-targeting'
test_perturb = tuple(config['fewshot'].items())[0][1]['test']
for 

In [ ]:
sampler.model.encoder.forward()

In [5]:
import torch

# 加载 checkpoint（如果显存有限可以用 map_location="cpu"）
ckpt = torch.load("/work/home/cryoem666/xyf/temp/pycharm/Squidiff/checkpoints/ours/model.pt",
                  map_location="cpu")

# 1. 如果是直接保存的 state_dict
if isinstance(ckpt, dict) and all(isinstance(v, torch.Tensor) for v in ckpt.values()):
    keys = ckpt.keys()
# 2. 如果 checkpoint 是个 dict，里头有 'state_dict' 或类似 key
elif isinstance(ckpt, dict) and "state_dict" in ckpt:
    keys = ckpt["state_dict"].keys()
else:
    # 其他情况，先看一下顶层 keys
    print("checkpoint 顶层 keys:", ckpt.keys())
    # 假设第一个 key 对应的就是参数 dict
    first = next(iter(ckpt.values()))
    if isinstance(first, dict):
        keys = first.keys()
    else:
        raise ValueError("无法识别 checkpoint 格式，请手动检查结构")

# 打印所有参数名
print("模型包含的权重层／参数名称：")
for k in keys:
    print(k)

模型包含的权重层／参数名称：
time_embed.0.weight
time_embed.0.bias
time_embed.2.weight
time_embed.2.bias
mlp_blocks.0.fc1.weight
mlp_blocks.0.fc1.bias
mlp_blocks.0.fc2.weight
mlp_blocks.0.fc2.bias
mlp_blocks.0.layer_norm1.weight
mlp_blocks.0.layer_norm1.bias
mlp_blocks.0.layer_norm2.weight
mlp_blocks.0.layer_norm2.bias
mlp_blocks.0.time_dense.weight
mlp_blocks.0.time_dense.bias
mlp_blocks.1.fc1.weight
mlp_blocks.1.fc1.bias
mlp_blocks.1.fc2.weight
mlp_blocks.1.fc2.bias
mlp_blocks.1.layer_norm1.weight
mlp_blocks.1.layer_norm1.bias
mlp_blocks.1.layer_norm2.weight
mlp_blocks.1.layer_norm2.bias
mlp_blocks.1.time_dense.weight
mlp_blocks.1.time_dense.bias
mlp_blocks.2.fc1.weight
mlp_blocks.2.fc1.bias
mlp_blocks.2.fc2.weight
mlp_blocks.2.fc2.bias
mlp_blocks.2.layer_norm1.weight
mlp_blocks.2.layer_norm1.bias
mlp_blocks.2.layer_norm2.weight
mlp_blocks.2.layer_norm2.bias
mlp_blocks.2.time_dense.weight
mlp_blocks.2.time_dense.bias
input_layer.weight
input_layer.bias
output_layer.weight
output_layer.bias
